# 2. Data Preprocessing - Gold Price Forecasting

**Objective:** Clean, transform, and prepare data for machine learning models.

**Author:** Félix Jouary  
**Dataset:** Kaggle Gold Price Dataset

## 2.1 Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')

print("Libraries imported successfully!")

## 2.2 Load Raw Data

In [ ]:
# Load the daily dataset
df = pd.read_csv('../data/raw/Daily.csv', na_values=['#N/A', '#N/A N/A', 'N/A'])

# Fix numeric columns - remove commas and convert to float
for col in df.columns:
    if col != 'Date':
        df[col] = df[col].replace({',': ''}, regex=True)
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Convert Date column
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

# Sort by date
df = df.sort_values('Date').reset_index(drop=True)

print(f"Original dataset shape: {df.shape}")
df.head()

## 2.3 Focus on USD Price (Target Variable)

For this project, we will focus on predicting the **Gold Price in USD**. Other currencies are highly correlated and would add complexity without significant value.

In [ ]:
# Create a clean dataframe with Date and USD only
df_gold = df[['Date', 'USD']].copy()
df_gold.columns = ['Date', 'Price']

print(f"Working dataset shape: {df_gold.shape}")
print(f"\nDate range: {df_gold['Date'].min()} to {df_gold['Date'].max()}")
print(f"\nMissing values: {df_gold['Price'].isnull().sum()}")
df_gold.head()

## 2.4 Handle Missing Values

In [ ]:
# Check for missing values
print("Missing values before treatment:")
print(df_gold.isnull().sum())

# For time series, forward fill is appropriate (use previous day's price)
df_gold['Price'] = df_gold['Price'].ffill()

# If any remaining NaN at the beginning, backward fill
df_gold['Price'] = df_gold['Price'].bfill()

print("\nMissing values after treatment:")
print(df_gold.isnull().sum())

# Check for duplicates
print("\n" + "="*50)
print("DUPLICATE CHECK")
print("="*50)
duplicates_date = df_gold[df_gold.duplicated(subset=['Date'], keep=False)]
duplicates_full = df_gold[df_gold.duplicated(keep=False)]

print(f"Duplicate dates: {len(duplicates_date)}")
print(f"Fully duplicate rows: {len(duplicates_full)}")

if len(duplicates_date) > 0:
    print("\nRemoving duplicate dates (keeping first occurrence)...")
    df_gold = df_gold.drop_duplicates(subset=['Date'], keep='first').reset_index(drop=True)
    print(f"Shape after removing duplicates: {df_gold.shape}")
else:
    print("No duplicates found - data is clean!")

## 2.5 Outlier Detection and Analysis

In [ ]:
# Calculate IQR for outlier detection
Q1 = df_gold['Price'].quantile(0.25)
Q3 = df_gold['Price'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df_gold[(df_gold['Price'] < lower_bound) | (df_gold['Price'] > upper_bound)]

print(f"IQR Method:")
print(f"Q1: {Q1:.2f}, Q3: {Q3:.2f}, IQR: {IQR:.2f}")
print(f"Lower bound: {lower_bound:.2f}, Upper bound: {upper_bound:.2f}")
print(f"\nNumber of outliers detected: {len(outliers)} ({len(outliers)/len(df_gold)*100:.2f}%)")

In [ ]:
# Visualize outliers
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Boxplot
axes[0].boxplot(df_gold['Price'].values)
axes[0].set_title('Boxplot - Gold Price (USD)')
axes[0].set_ylabel('Price (USD)')

# Time series with outliers highlighted
axes[1].plot(df_gold['Date'], df_gold['Price'], label='Price', alpha=0.7)
axes[1].scatter(outliers['Date'], outliers['Price'], color='red', label='Outliers', s=10)
axes[1].set_title('Gold Price with Outliers Highlighted')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Price (USD)')
axes[1].legend()

plt.tight_layout()
plt.savefig('../reports/figures/outliers_analysis.png', dpi=150)
plt.show()

In [ ]:
# IMPORTANT: For gold prices, "outliers" are actually valid high prices during bull markets
# We should NOT remove them as they represent real market conditions
# This is a key insight for financial time series

print("Decision: Keep all data points.")
print("Reason: In financial time series, extreme values are valid market conditions,")
print("not measurement errors. Removing them would bias our model.")

## 2.6 Feature Engineering for Time Series

Creating features that capture temporal patterns and financial indicators.

In [ ]:
# Make a copy for feature engineering
df_features = df_gold.copy()

# 1. Temporal Features
df_features['Year'] = df_features['Date'].dt.year
df_features['Month'] = df_features['Date'].dt.month
df_features['Day'] = df_features['Date'].dt.day
df_features['DayOfWeek'] = df_features['Date'].dt.dayofweek
df_features['Quarter'] = df_features['Date'].dt.quarter
df_features['WeekOfYear'] = df_features['Date'].dt.isocalendar().week.astype(int)

print("Temporal features created!")
df_features.head()

In [ ]:
# 2. Lag Features (previous days' prices)
for lag in [1, 2, 3, 5, 7, 14, 21, 30]:
    df_features[f'Price_Lag_{lag}'] = df_features['Price'].shift(lag)

print("Lag features created!")
print(f"Columns: {[col for col in df_features.columns if 'Lag' in col]}")

In [ ]:
# 3. Moving Averages (trend indicators)
for window in [7, 14, 30, 60, 90]:
    df_features[f'MA_{window}'] = df_features['Price'].rolling(window=window).mean()

print("Moving average features created!")
print(f"Columns: {[col for col in df_features.columns if 'MA_' in col]}")

In [ ]:
# 4. Price Returns (percentage change)
df_features['Return_1d'] = df_features['Price'].pct_change(1) * 100
df_features['Return_5d'] = df_features['Price'].pct_change(5) * 100
df_features['Return_30d'] = df_features['Price'].pct_change(30) * 100

print("Return features created!")

In [ ]:
# 5. Volatility (rolling standard deviation)
df_features['Volatility_7d'] = df_features['Return_1d'].rolling(window=7).std()
df_features['Volatility_30d'] = df_features['Return_1d'].rolling(window=30).std()

print("Volatility features created!")

In [ ]:
# 6. Price momentum indicators
df_features['Price_vs_MA7'] = (df_features['Price'] / df_features['MA_7'] - 1) * 100
df_features['Price_vs_MA30'] = (df_features['Price'] / df_features['MA_30'] - 1) * 100

# 7. Rolling min/max (support/resistance levels)
df_features['Rolling_Min_30'] = df_features['Price'].rolling(window=30).min()
df_features['Rolling_Max_30'] = df_features['Price'].rolling(window=30).max()

print("Momentum and support/resistance features created!")

In [ ]:
# Check all features
print(f"Total features: {len(df_features.columns)}")
print(f"\nAll columns:")
print(df_features.columns.tolist())

In [ ]:
# Check missing values created by feature engineering
print("Missing values after feature engineering:")
missing = df_features.isnull().sum()
print(missing[missing > 0])

## 2.7 Handle Missing Values from Feature Engineering

In [ ]:
# Drop rows with NaN (first ~90 days due to rolling windows)
df_clean = df_features.dropna().reset_index(drop=True)

print(f"Shape before cleaning: {df_features.shape}")
print(f"Shape after cleaning: {df_clean.shape}")
print(f"Rows removed: {len(df_features) - len(df_clean)}")

# Verify no missing values
print(f"\nRemaining missing values: {df_clean.isnull().sum().sum()}")

## 2.8 Define Target Variable

We will predict the **next day's price** (Price at t+1).

In [ ]:
# Create target: next day's price
df_clean['Target'] = df_clean['Price'].shift(-1)

# Remove last row (no target available)
df_clean = df_clean.dropna().reset_index(drop=True)

print(f"Final dataset shape: {df_clean.shape}")
print(f"\nTarget variable: Price at t+1")
print(f"Target range: {df_clean['Target'].min():.2f} - {df_clean['Target'].max():.2f}")

## 2.9 Train-Test Split (Time Series)

**Important:** For time series, we must respect temporal order. No random shuffling!

In [ ]:
# Define split ratio (80% train, 20% test)
train_size = 0.8
split_idx = int(len(df_clean) * train_size)

# Split data
train_data = df_clean.iloc[:split_idx].copy()
test_data = df_clean.iloc[split_idx:].copy()

print(f"Training set: {len(train_data)} samples ({len(train_data)/len(df_clean)*100:.1f}%)")
print(f"Test set: {len(test_data)} samples ({len(test_data)/len(df_clean)*100:.1f}%)")
print(f"\nTraining period: {train_data['Date'].min()} to {train_data['Date'].max()}")
print(f"Test period: {test_data['Date'].min()} to {test_data['Date'].max()}")

In [ ]:
# Visualize train-test split
plt.figure(figsize=(14, 5))
plt.plot(train_data['Date'], train_data['Price'], label='Training Set', color='blue')
plt.plot(test_data['Date'], test_data['Price'], label='Test Set', color='orange')
plt.axvline(x=train_data['Date'].iloc[-1], color='red', linestyle='--', label='Split Point')
plt.title('Train-Test Split Visualization')
plt.xlabel('Date')
plt.ylabel('Gold Price (USD)')
plt.legend()
plt.tight_layout()
plt.savefig('../reports/figures/train_test_split.png', dpi=150)
plt.show()

## 2.10 Feature Scaling

In [ ]:
# Define feature columns (exclude Date and Target)
feature_cols = [col for col in df_clean.columns if col not in ['Date', 'Target']]

print(f"Number of features: {len(feature_cols)}")
print(f"\nFeatures: {feature_cols}")

In [ ]:
# Prepare X and y
X_train = train_data[feature_cols].values
y_train = train_data['Target'].values

X_test = test_data[feature_cols].values
y_test = test_data['Target'].values

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

In [ ]:
# Scale features using StandardScaler
# IMPORTANT: Fit only on training data to avoid data leakage!
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Features scaled using StandardScaler")
print(f"\nTraining set - Mean: {X_train_scaled.mean():.6f}, Std: {X_train_scaled.std():.6f}")
print(f"Test set - Mean: {X_test_scaled.mean():.6f}, Std: {X_test_scaled.std():.6f}")

## 2.11 Save Processed Data

In [ ]:
# Save processed datasets
train_data.to_csv('../data/processed/train_data.csv', index=False)
test_data.to_csv('../data/processed/test_data.csv', index=False)

# Save full processed dataset
df_clean.to_csv('../data/processed/gold_features.csv', index=False)

# Save scaled arrays for modeling
np.save('../data/processed/X_train_scaled.npy', X_train_scaled)
np.save('../data/processed/X_test_scaled.npy', X_test_scaled)
np.save('../data/processed/y_train.npy', y_train)
np.save('../data/processed/y_test.npy', y_test)

# Save feature names
pd.Series(feature_cols).to_csv('../data/processed/feature_names.csv', index=False)

# Save scaler for later use
import joblib
joblib.dump(scaler, '../data/processed/scaler.pkl')

print("All processed data saved successfully!")
print("\nFiles created:")
print("- ../data/processed/train_data.csv")
print("- ../data/processed/test_data.csv")
print("- ../data/processed/gold_features.csv")
print("- ../data/processed/X_train_scaled.npy")
print("- ../data/processed/X_test_scaled.npy")
print("- ../data/processed/y_train.npy")
print("- ../data/processed/y_test.npy")
print("- ../data/processed/feature_names.csv")
print("- ../data/processed/scaler.pkl")

## 2.12 Summary

### Preprocessing Steps Completed:

1. **Data Loading:** Loaded daily gold price dataset
2. **Missing Values:** Handled using forward-fill method
3. **Outlier Analysis:** Identified but kept (valid market data)
4. **Feature Engineering:**
   - Temporal features (Year, Month, Day, DayOfWeek, Quarter)
   - Lag features (1, 2, 3, 5, 7, 14, 21, 30 days)
   - Moving averages (7, 14, 30, 60, 90 days)
   - Returns (1d, 5d, 30d)
   - Volatility (7d, 30d)
   - Momentum indicators
5. **Target Variable:** Next day's price (t+1)
6. **Train-Test Split:** 80/20 chronological split
7. **Feature Scaling:** StandardScaler (fit on train only)

### Ready for Modeling!

In [ ]:
# Final summary
print("="*50)
print("PREPROCESSING SUMMARY")
print("="*50)
print(f"Original samples: {len(df)}")
print(f"Final samples: {len(df_clean)}")
print(f"Number of features: {len(feature_cols)}")
print(f"Training samples: {len(train_data)}")
print(f"Test samples: {len(test_data)}")
print(f"\nTarget: Gold Price at t+1 (USD)")
print("="*50)